# Linear Regression

### 1. import libraries

In [ ]:

import "fmt"
import "gonum.org/v1/gonum/blas/blas64"
import "gonum.org/v1/gonum/mat"
import "gonum.org/v1/gonum/stat"
import "gonum.org/v1/plot"
import "gonum.org/v1/plot/plotter"
import "gonum.org/v1/plot/plotutil"
import "gonum.org/v1/plot/vg"
import "strings"
import "strconv"
import "github.com/kniren/gota/dataframe"
import "os"
import "encoding/csv"
import "github.com/kniren/gota/series"
import "math"
import "image/color"
import "io/ioutil"

In [ ]:

func GetGraph(graphName string) ([]byte, error) { //utility to load graphs (to use later)
	infile, err := os.Open(graphName)
	bytes, err := ioutil.ReadAll(infile)
	infile.Close()
	return bytes, err
}

#### 2. Load working csv file

In [ ]:
file, err := os.Open("../data/basketball.heigh.and.weigh.csv")
if err != nil {
    fmt.Println(err)
}

In [ ]:
dataset := dataframe.ReadCSV(file)

In [ ]:
file.Close()

In [ ]:
fmt.Println(dataset)

### We can select one or many columns of a dataFrame :

In [ ]:
fmt.Println(dataset.Col("Team").Subset([]int{1,2,3,4,5}))

In [ ]:
fmt.Println(dataset.Select([]string{"Name","Position"}))

##### select name, position and heigt for the Chicago White Sox 

In [ ]:
filter := dataframe.F{
    Colname:    "Team",
    Comparator: series.Eq,
    Comparando: "CWS",
}

In [ ]:
fmt.Println(dataset.Filter(filter).Select([]string{"Name","Position","Weight(pounds)"}))

### Find the average width and heigh by team

find all teams (no duplicated)

In [ ]:
teams :=dataset.Col("Team").Records()

In [ ]:
teamsMap := make(map[string]string)
for _,val := range teams {//build a map with team names
    teamsMap[val]=val
}

In [ ]:
fmt.Println(teamsMap)

For each team, find the average 

In [ ]:
for key :=range teamsMap{
    filter := dataframe.F{
    Colname:    "Team",
    Comparator: series.Eq,
    Comparando: key,
}
    fmt.Println(key, stat.Mean(dataset.Filter(filter).Col("Weight(pounds)").Float(),nil),stat.Mean(dataset.Filter(filter).Col("Height(inches)").Float(),nil) )
}

#### Visualize the relationship between width and heigh

In [ ]:
plotGrid, err := plot.New()
if err != nil {
    fmt.Println(err)
}

In [ ]:
plotGrid.Title.Text = "relationship between width and heigh"
plotGrid.Y.Label.Text = "Weight"
plotGrid.X.Label.Text = "Height"

In [ ]:
plotGrid.Add(plotter.NewGrid())

In [ ]:

widthsHeighs := dataset.Select([]string{"Height(inches)","Weight(pounds)"})


In [ ]:
fmt.Println(widthsHeighs)

In [ ]:
ptsPlot := make(plotter.XYs, widthsHeighs.Nrow())
for i,val := range ptsPlot { // builds points to add to our graph
    ptsPlot[i].X = widthsHeighs.Elem(i,0).Float()
    ptsPlot[i].Y = widthsHeighs.Elem(i,1).Float()
    if widthsHeighs.Elem(i,0).IsNA() {
        ptsPlot[i].X = 0
    }
    if widthsHeighs.Elem(i,1).IsNA() {
        ptsPlot[i].Y = 0
    }
}

In [ ]:
s, err := plotter.NewScatter(ptsPlot)
if err != nil {
    fmt.Println(err)
}

In [ ]:
s.GlyphStyle.Color = color.RGBA{R: 255, B: 128, A: 255}

In [ ]:
plotGrid.Add(s)

In [ ]:
if err := plotGrid.Save(4*vg.Inch, 4*vg.Inch, "wvsh1.png"); err != nil {
	fmt.Println(err)
}


In [ ]:
graph1, err :=GetGraph("wvsh1.png")
if err != nil {
    fmt.Println(err)
}

In [ ]:
display.PNG(graph1)

Find the linear regression of this data. The idea is to find the best-fit line: y = alpha + beta*x

In [ ]:
alpha, beta := stat.LinearRegression(widthsHeighs.Col("Height(inches)").Float(),widthsHeighs.Col("Weight(pounds)").Float(), nil, false)

In [ ]:
fmt.Printf("y = %f + %f * x",alpha,beta) 